In [1]:
# Import the necessary modules
import pandas as pd
import numpy as np
import scipy
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn import linear_model
from sklearn import preprocessing
import statsmodels.api as sm
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVR
from sklearn.svm import SVC
from sklearn import ensemble
from sklearn import datasets
from sklearn.utils import shuffle
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import cross_val_score

C:\Users\maken\Anaconda3\lib\site-packages\statsmodels\compat\pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [2]:
AAPL = pd.read_csv('D:\\Data\\sandp500\\AAPL_data.csv')

In [3]:
AAPL.tail()

,Date,Open,High,Low,Close,Volume,Name
1253,2017-08-07,157.06,158.92,156.67,158.81,21870321,AAPL
1254,2017-08-08,158.60,161.83,158.27,160.08,36205896,AAPL
1255,2017-08-09,159.26,161.27,159.11,161.06,26131530,AAPL
1256,2017-08-10,159.90,160.00,154.63,155.32,40804273,AAPL
1257,2017-08-11,156.60,158.57,156.07,157.48,26257096,AAPL


In [4]:
# The momentum will show how many days in a row the stock has moved up or down. 

# Create a list to store the momentum
momentum = [0]
i=1
# Calculate the momentums and store them in the new column, 'Momentum'
for row in AAPL['Close']:
    if i < len(AAPL):
        if AAPL.Close[i] > AAPL.Close[i-1]:
            momentum.append(+1)
            i = i+1
        elif AAPL.Close[i] < AAPL.Close[i-1]:
            momentum.append(-1)
            i = i+1

print(len(momentum))
AAPL['Momentum'] = momentum
AAPL.head()

414


ValueError: Length of values does not match length of index

In [ ]:
streak = [0] * len(AAPL)
i=1
# Calculate the streaks and store them in the new column, 'Streak'
for row in AAPL['Close']:
    if i < len(AAPL):
        if AAPL.Close[i] > AAPL.Close[i-1]:
            if streak[i-1] >= 0:
                streak[i] = streak[i-1]+1
                i = i+1
            else:
                streak[i]=0
                i = i+1
        elif AAPL.Close[i] < AAPL.Close[i-1]:
            if streak[i-1] <= 0:
                streak[i] = streak[i-1]-1
                i = i+1
            else:
                streak[i]=0
                i = i+1

AAPL['Streak'] = streak
AAPL.head()

In [ ]:
# Create a 'Future Momentum' feature that the model will attempt to predict.
AAPL['Future Momentum'] = AAPL.Momentum.shift(-1)
AAPL.head()# Drop the last row to get rid of the NaN values
AAPL = AAPL.iloc[:len(data)-1,:]

In [ ]:
# Set the variables. 
# Use the closing value for Y
# Use the new features for X
Y = AAPL['Future Momentum']
X = AAPL[['Close', 'Volume', 'Momentum', 'Streak']]

# Create training and test sets.
offset = int(X.shape[0] * 0.8)

# Put 90% of the data in the training set.
X_train, Y_train = X[:offset], Y[:offset]

# And put 10% in the test set.
X_test, Y_test = X[offset:], Y[offset:]

In [ ]:
# Run the pred_return function for ridge
y_pred = ridgeregr.fit(X_train,Y_train).predict(X_test)
cash_available = 0
stocks_owned = 0
values = pred_return(y_pred, data, cash_available, stocks_owned)

In [ ]:
# Calculate the ending returns from the lr model
ridgeregr_returns = values.loc[len(values)-1,'Stocks']*data.loc[len(y_pred)-1,'Close'] + values.loc[len(values)-1, 'Cash']
print(ridgeregr_returns)

In [ ]:
# 252 trading days in a year, 2017-2012 is 6 years, 
# $1000 granted every 10 days = 151,200 dollars granted.
# Since we are predicting the last 20% of the data, divide by 20
# Get total profit by subrtracting returns by dollars granted. 
ridgeregr_profit = ridgeregr_returns - 7560
print(ridgeregr_profit)

# Store the profit in the model_profits table
profits = {'Model':['Ridge Regression'], 'Profit':ridgeregr_profit}
model_profits = model_profits.append(pd.DataFrame(data=profits, columns=profits.keys()), ignore_index=True)